In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json

import alphagogoat as ag
import alphagogoat.utils
import re

In [3]:
a = ag.utils.make_battle("./history/historical_battle.json", "./history/historical_teams.json")
len(a)
a

AttributeError: module 'alphagogoat.utils' has no attribute 'make_battle'

In [ ]:
from poke_env.player_configuration import PlayerConfiguration
from poke_env.environment.battle import Battle

with open("./history/historical_battle.json") as f:
    battle_data = json.load(f)

#battle_data

battle_data

In [ ]:
pattern = r'\[Gen ([89])\]'
gen = int(re.findall(pattern, battle_data['format'])[0])
#print(int(matches[0]))

MESSAGES_TO_IGNORE = {"", "t:", "expire"}

battle = Battle(battle_data['id'], battle_data['p1'], battle_data['log'], gen)
battle._opponent_username = battle_data['p2']

In [ ]:
battle.opponent_username


In [ ]:
import pypokedex

p= pypokedex.get(name = 'Indeedee')
p.dex

In [ ]:
import logging
from poke_env.environment.pokemon import Pokemon, PokemonType
import pypokedex
from poke_env.environment.weather import Weather
from poke_env.environment.move import Move
from poke_env.environment.side_condition import SideCondition
from poke_env.environment.status import Status
import copy

logger = logging.getLogger('poke-env')
with open("cache/replays/gen8randombattle-1862087705.json") as f:
    battle_data = json.load(f)

history = battle_data['log']

history = history.split('\n')

curr_p1_pokemon = None
curr_p2_pokemon = None

curr_battle = Battle(battle_data['id'], battle_data['p1'],logger, gen)

generation_dict = {}

gen1 = set([i for i in range(1, 152)])
gen2 = set([i for i in range(152, 252)])
gen3 = set([i for i in range(252, 387)])
gen4 = set([i for i in range(387, 494)])
gen5 = set([i for i in range(494, 650)])
gen6 = set([i for i in range(650, 722)])
gen7 = set([i for i in range(722, 810)])
gen8 = set([i for i in range(810, 906)])

for i in range(1, 906):
    if i in gen1:
        generation_dict[i] = 1
    elif i in gen2:
        generation_dict[i] = 2
    elif i in gen3:
        generation_dict[i] = 3
    elif i in gen4:
        generation_dict[i] = 4
    elif i in gen5:
        generation_dict[i] = 5
    elif i in gen6:
        generation_dict[i] = 6
    elif i in gen7:
        generation_dict[i] = 7
    elif i in gen8:
        generation_dict[i] = 8

to_skip = []
battle_log = []

opponent_active_pokemon = None

for idx, line in enumerate(history):
    if len(line) <= 1 or idx in to_skip:
        continue
    
    if line.startswith("|start"): # start of battle
        #print(line)
        switch_regex_pattern1 = r"p1a:\s(.*?)\|.*?L(\d+).*?(M|F)\|(\d+)/(\d+)"
        switch_regex_pattern2 = r"p2a:\s(.*?)\|.*?L(\d+).*?(M|F)\|(\d+)/(\d+)"
        genderless_1 = r"p1a:\s(.*?)\|.*?L(\d+).*?(?:M|F)?\|(\d+)/(\d+)"
        genderless_2 = r"p2a:\s(.*?)\|.*?L(\d+).*?(?:M|F)?\|(\d+)/(\d+)"
        p1_info = history[idx + 1]
        p2_info = history[idx + 2]
        to_skip.append(idx + 1)
        to_skip.append(idx + 2)
        match1 = re.search(switch_regex_pattern1, p1_info)
        match2 = re.search(switch_regex_pattern2, p2_info)
        match3 = re.search(genderless_1, p1_info)
        match4 = re.search(genderless_2, p2_info)
        
        if match1:
            pokemon_name = match1.group(1)
            level = int(match1.group(2))
            gender = match1.group(3)
            curr_hp = int(match1.group(4))
            max_hp = int(match1.group(5))

        elif match3:
            pokemon_name = match3.group(1)
            level = int(match3.group(2))
            curr_hp = int(match3.group(3))
            max_hp = int(match3.group(4))
            gender = None
        
        else:
            line = p1_info.split('|')
            player_number = int(line[2][1])
            pokemon_name = line[2].split(' ')[1]
            level = int(line[3].split(' ')[1][1:])
            hp_numerator = int(line[4].split('/')[0])
            hp_denominator = int(line[4].split('/')[1])
        
        curr_p1_pokemon = Pokemon(gen = 8, species = pokemon_name) #pokemon1_info.name)

        curr_battle._team[pokemon_name.replace(" ","").lower()] = curr_p1_pokemon
        curr_battle._team[pokemon_name.replace(" ","").lower()]._active = True

        curr_p1_pokemon._level = level
        curr_p1_pokemon._active = True
        curr_p1_pokemon._current_hp = curr_hp
        curr_p1_pokemon._max_hp = max_hp
        curr_p1_pokemon._first_turn = True
        curr_p1_pokemon._status = None
        
        if match2:
            #print("match2")
            #print('here')
            pokemon_name = match2.group(1)
            level = int(match2.group(2))
            gender = match2.group(3)
            curr_hp = int(match2.group(4))
            max_hp = int(match2.group(5))
        elif match4:
            pokemon_name = match4.group(1)
            level = int(match4.group(2))
            curr_hp = int(match4.group(3))
            max_hp = int(match4.group(4))
        else:
            line = p2_info.split('|')
            player_number = int(line[2][1])
            pokemon_name = line[2].split(' ')[1]
            level = int(line[3].split(' ')[1][1:])
            hp_numerator = int(line[4].split('/')[0])
            hp_denominator = int(line[4].split('/')[1])

        curr_p2_pokemon = Pokemon(gen = 8, species = pokemon_name)
        curr_battle._opponent_team[pokemon_name.replace(" ","").lower()] = curr_p2_pokemon
        curr_battle._opponent_team[pokemon_name.replace(" ","").lower()]._active = True

        curr_p2_pokemon._level = level
        curr_p2_pokemon._current_hp = curr_hp
        curr_p2_pokemon._max_hp = max_hp
        curr_p2_pokemon._first_turn = True
        curr_p2_pokemon._status = None
        # else:
        #     print("no match")
        
    if line.startswith("|-weather|"):
        weather_regex = r"^\|-weather\|(\w+)\|\[from\]\s(\w+):\s(.*?)\|\[of\]\sp(\d+)a:\s(.*?)$"

        match = re.search(weather_regex, line)

        if 'none' in line:
            curr_battle._weather = None
            continue
        if match:
            weather = match.group(1)
            source_type = match.group(2)
            source = match.group(3)
            player = int(match.group(4))
            pokemon_name = match.group(5)
            curr_battle._weather = Weather.from_name(weather)
        
        

    if line.startswith("|switch|"):
        switch_regex_pattern = r'\|switch\|p(\d)a: (\w+)\|\w+, L(\d+), \w+\|(\d+)/(\d+)'
        match1 = re.search(switch_regex_pattern, line)

        if match1:
            player_number = int(match1.group(1))
            pokemon_name = match1.group(2)
            level = match1.group(3)
            hp_numerator = match1.group(4)
            hp_denominator = match1.group(5)
        else:
            line = line.split('|')
            player_number = int(line[2][1])
            pokemon_name = line[2].split(' ')[1]
            level = int(re.sub(r'\D', '', line[3].split(' ')[1][1:]))
            hp_numerator = int(line[4].split('/')[0])
            hp_denominator = int(line[4].split('/')[1])

        if player_number == 1:
            if pokemon_name not in curr_battle.team:

                curr_battle._team[curr_p1_pokemon._species]._active = False

                curr_p1_pokemon = Pokemon(gen = 8, species = pokemon_name)

                curr_battle._team[pokemon_name.replace(" ","").lower()] = curr_p1_pokemon
                curr_battle._team[pokemon_name.replace(" ","").lower()]._active = True

                #curr_battle.active_pokemon = curr_p1_pokemon

                curr_p1_pokemon._level = level
                curr_p1_pokemon._active = True
                curr_p1_pokemon._current_hp = curr_hp
                curr_p1_pokemon._max_hp = max_hp
                curr_p1_pokemon._first_turn = True
                curr_p1_pokemon._status = None

            else:
                curr_battle.active_pokemon = curr_battle.team[pokemon_name]
        else:
            
            #curr_battle.opponent_active_pokemon._active = False
            if pokemon_name not in curr_battle.opponent_team:

                curr_battle._opponent_team[curr_p2_pokemon._species]._active = False

                curr_p2_pokemon = Pokemon(gen = 8, species = pokemon_name)
                curr_battle._opponent_team[pokemon_name.replace(" ","").lower()] = curr_p2_pokemon
                curr_battle._opponent_team[pokemon_name.replace(" ","").lower()]._active = True

                curr_p2_pokemon._level = level
                curr_p2_pokemon._active = True
                curr_p2_pokemon._current_hp = curr_hp
                curr_p2_pokemon._max_hp = max_hp
                curr_p2_pokemon._first_turn = True
                curr_p2_pokemon._status = None
            
            else:
                #curr_battle.active_pokemon = curr_battle.opponent_team[pokemon_name.replace(" ","").lower()]
                pass

    if '|-damage|' in line:
        if 'fnt' not in line:
        #print(line)
            line = line.split('|')
            player = int(line[2][1])
            curr_hp = int(line[3].split('/')[0])
            if player == 1:
                curr_battle.active_pokemon._current_hp = curr_hp
            else:
                curr_battle.opponent_active_pokemon._current_hp = curr_hp
        

    #if line.startswith("|-ability|"):
    if '|-ability|' in line:
        ability_regex = r"p(\d+)a:\s(.*?)\|.*\|(\w+)\|(\w+)$"

        match = re.search(ability_regex, line)

        if match:
            player = int(match.group(1))
            pokemon_name = match.group(2)
            ability = match.group(3)
            ability_effect = match.group(4)

            if player == 1:
                curr_battle.active_pokemon.ability = ability
                curr_battle.active_pokemon.ability_effect = ability_effect
            else:
                curr_battle.opponent_active_pokemon.ability = ability
                curr_battle.opponent_active_pokemon.ability_effect = ability_effect

    #if line.startswith("|move|"):
    if '|move|' in line:
        
        line = line.split('|')
        
        player_number = line[2][1]
        move = line[3].replace(' ', '').lower()
        move = re.sub(r'\W+', '', move).lower()

        if player_number == '1':
            if move not in curr_battle.active_pokemon.moves:
                curr_battle.active_pokemon.moves[move] = Move(move, gen=8)
            curr_battle.active_pokemon.moves[move].use()
        else:
            #if move not in curr_battle.opponent_active_pokemon.moves:
            curr_battle.opponent_active_pokemon.moves[move] = Move(move, gen=8)
            curr_battle.opponent_active_pokemon.moves[move].use()

    #if line.startswith("|-heal|"):
    if "|-heal|" in line:
        heal = line.split('|')
        player = int(heal[2][1])
        pokemon_name = heal[2].split(' ')[1]
        curr_hp = int(heal[3].split('/')[0])
        max_hp = int(heal[3].split('/')[1].split(' ')[0])

        if player == 1:
            curr_battle.active_pokemon._current_hp = curr_hp
        else:
            curr_battle.opponent_active_pokemon._current_hp = curr_hp

    #if line.startswith("|-sidestart|"): # hazards
    if "|-sidestart|" in line:
        hazard_message = line.split('|')[3]
        player = line.split('|')[2][1]

        if player == 1:
            curr_battle.side_conditions[SideCondition.from_showdown_message(hazard_message)] = True
        else:
            curr_battle.opponent_side_conditions[SideCondition.from_showdown_message(hazard_message)] = True

    if "|-boost|" in line:
        boost_regex = r"p(\d+)a:\s\w+\|(\w+)\|(\d+)"

        match = re.search(boost_regex, line)
        player = int(match.group(1))
        stat = match.group(2)
        amount = int(match.group(3))

        if player == 1:
            curr_battle.active_pokemon.boosts[stat] += amount
        else:
            curr_battle.opponent_active_pokemon.boosts[stat] += amount

    #if line.startswith("|-unboost|"):
    if "|-unboost|" in line:
        boost_regex = r"p(\d+)a:\s\w+\|(\w+)\|(\d+)"

        match = re.search(boost_regex, line)
        player = int(match.group(1))
        stat = match.group(2)
        amount = int(match.group(3))

        if player == 1:
            curr_battle.active_pokemon.boosts[stat] -= amount
        else:
            curr_battle.opponent_active_pokemon.boosts[stat] -= amount


    #if line.startswith("|faint|"):
    if "|faint|" in line:
        faint_regex = r"p(\d+)a:\s(.*?)$"

        match = re.search(faint_regex, line)
        player = int(match.group(1))
        pokemon_name = match.group(2)

        if player == 1:
            curr_battle.active_pokemon._status = Status.FNT
            curr_battle.active_pokemon._active = False
            
        else:
            curr_battle.opponent_active_pokemon._status = Status.FNT
            curr_battle.opponent_active_pokemon._active = False
            

    #if line.startswith("|-status|"):
    if "|-status|" in line:
        status_regex = r"p(\d+)a:\s(.*?)\|(\w+)"

        match = re.search(status_regex, line)
        player = int(match.group(1))
        pokemon_name = match.group(2)
        status = match.group(3)

        if status == 'par':
            status = Status.PAR
        elif status == 'brn':
            status = Status.BRN
        elif status == 'psn':
            status = Status.PSN
        elif status == 'tox':
            status = Status.TOX
        elif status == 'slp':
            status = Status.SLP
        elif status == 'frz':
            status = Status.FRZ
        
        if player == 1:
            curr_battle.active_pokemon._status = status
        else:
            curr_battle.opponent_active_pokemon._status = status
    
    if "|-sideend|" in line:
        hazard_message = line.split('|')[3]
        player = line.split('|')[2][1]

        if player == 1:
            curr_battle.side_conditions[SideCondition.from_showdown_message(hazard_message)] = 0
        else:
            curr_battle.opponent_side_conditions[SideCondition.from_showdown_message(hazard_message)] = 0

    battle_log.append(copy.deepcopy(curr_battle))


In [ ]:
s = r"p(\d+)a:\s(.*?)\|(\d+)/(\d+)$"
t = r'\|switch\|p(\d)a: (\w+)\|\w+, L(\d+), \w+\|(\d+)/(\d+)'
u = r"p2a:\s(.*?)\|.*?L(\d+).*?(?:M|F)?\|(\d+)/(\d+)"
v = r'\|switch\|p(\d)a: (\w+)\|\w+, L(\d+), \w+\|(\d+)/(\d+)'
#w = r"\|switch\|p(\d+)a: (\w+)(?<=\|\w+\|\w+:\s)(\w+)\|(\d+)/(\d+)"
x = r"\|switch\|p(\d+)a: \w+\|.*?, L(\d+),.*?\|(\d+)/(\d+)"

# match = re.search(s, "-damage|p1a: Sudowoodo|103/275")
# print(match.group(4))
# match = re.search(t, "|switch|p2a: Heliolisk|Heliolisk, L82, M|236/236")
# match.group(1)
# match.group(2)
# match.group(3)
# match.group(4)
# match.group(5)
# match = re.search(u, "|switch|p2a: Palkia|Palkia, L74|255/255")
# match.group(1)
# match.group(2)
# match.group(3)
# match.group(4)

match = re.search(x,"|switch|p2a: Zacian|Zacian-Crowned, L62|218/218")
match.group(1)
match == None

In [ ]:
p = Pokemon(gen = 8, species = 'Indeedee')
p._type_2

In [ ]:
#print(history[24])
for idx, line in enumerate(history):#.split('\n'):
    if line.startswith("|-sidestart|"):
        print(line)
        # print(history[idx + 1])
        # print(history[idx + 2])
    # if 'defog' in line:
    #     print(line)

In [ ]:
for line in battle_data['inputlog'].split('\n'):
    print(line)